In [1]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle
import time

import data_utils
import model_utils_train
import model_utils_infer
import train_utils
import evaluation
importlib.reload(data_utils)
importlib.reload(model_utils_train)
importlib.reload(model_utils_infer)


importlib.reload(train_utils)
importlib.reload(evaluation)

import torch
import torch.nn as nn

from transformers import DistilBertPreTrainedModel
from transformers import DistilBertForQuestionAnswering
from transformers import DistilBertConfig, DistilBertTokenizer

### Load data

In [2]:
file_dict = {"train1.1tiny": "./data/tiny_train-v1.1.json",
             "train2.0tiny": "./data/tiny_train-v2.0.json",
             "train1.1": "./data/train-v1.1.json", 
             "dev1.1": "./data/dev-v1.1.json"}
train_data_v1_debug = data_utils.prep_data(file_dict["train1.1tiny"])
train_data_v2_debug = data_utils.prep_data(file_dict["train2.0tiny"])
dev_data_v1 = data_utils.prep_data(file_dict["dev1.1"])
train_data_v1 = data_utils.prep_data(file_dict["train1.1"])

### Run inference with Huggingface pretrained BERT fine-tuned on SQuAD

In [ ]:
importlib.reload(model_utils)
importlib.reload(train_utils)
m = model_utils.PreTrainedSQuAD()
result = train_utils.inference(train_data_v2, m)

### Run inference with Huggingface pretrained DistillBERT fine-tuned on SQuAD

In [ ]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(train_data_v1, m)

In [ ]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(dev_data_v1, m)

### Train and run simple DistilBERT model

In [8]:
importlib.reload(model_utils_train)
importlib.reload(train_utils)
m = model_utils_train.SimpleDistilBERT()
result = train_utils.train(train_data=train_data_v1_debug, val_data=train_data_v1_debug, model=m, 
                           patience=10, lr=3e-5, print_freq=10)

Epoch: 0, LR: 3e-05, Train Loss: 10.5282, Val Loss: 9.8157, Val f1 7.931, epoch time: 1.7s
Epoch: 10, LR: 3e-05, Train Loss: 4.3554, Val Loss: 4.1090, Val f1 21.738, epoch time: 1.5s
Epoch: 20, LR: 3e-05, Train Loss: 3.0646, Val Loss: 3.2321, Val f1 40.183, epoch time: 1.5s
Epoch: 30, LR: 3e-05, Train Loss: 1.2089, Val Loss: 0.8216, Val f1 83.022, epoch time: 1.6s
Epoch: 40, LR: 3e-05, Train Loss: 0.4165, Val Loss: 0.2666, Val f1 86.909, epoch time: 1.5s
Epoch: 50, LR: 3e-05, Train Loss: 0.4213, Val Loss: 0.1705, Val f1 88.320, epoch time: 1.5s


In [ ]:
importlib.reload(model_utils_train)
m = model_utils_train.SimpleDistilBERT()
result = train_utils.train(train_data=train_data_v1, val_data=dev_data_v1, model=m, 
                           patience=10, lr=3e-5, print_freq=1)